In [ ]:
%stop_session

# Setup

In [ ]:
%iam_role IAM_ROLE_ARN
%region us-east-1
%glue_version 4.0
%idle_timeout 120
%number_of_workers 2
%extra_jars "s3://crawler-public/json/serde/json-serde.jar"
%spark_conf spark.sql.catalog.catalog.warehouse=s3://<your-warehouse-dir> --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions --conf spark.driver.maxResultSize=2g --conf spark.sql.catalog.catalog=org.apache.iceberg.spark.SparkCatalog --conf spark.sql.catalog.catalog.catalog-impl=org.apache.iceberg.aws.glue.GlueCatalog --conf spark.sql.catalog.catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog --conf spark.delta.logStore.class=org.apache.spark.sql.delta.storage.S3SingleDriverLogStore --conf spark.sql.shuffle.partitions=100
%%configure
{
    "datalake-formats": "iceberg,delta"
}



In [ ]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import com.amazonaws.services.glue.GlueContext
import com.amazonaws.services.glue.util.JsonOptions


In [ ]:
import org.apache.hadoop.fs._
import java.net._

def listPaths(path: String): List[String] = {
		val conf = spark.sparkContext.hadoopConfiguration
		
		FileSystem
			.get(URI.create(path), conf)
			.listStatus(new Path(path + "/"))
			.map(_.getPath.toString)
			.toList
	}

def mostRecentPath(path: String): String = {
    val conf = spark.sparkContext.hadoopConfiguration
    
    FileSystem
      .get(URI.create(path), conf)
      .listStatus(new Path(path + "/"))
      .last
      .getPath
      .toString
  }
